# Leitura dos arquivos formatados

## Descrição do notebook

O objetivo deste notebook é carregar os arquivos com os dados tratados do Pará e do Paraná para, em seguida, gerar um arquivo único com dados dos dois estados.

## Sumário

1. [Importação dos módulos e pacotes necessários](#importacao-dos-modulos-e-pacotes-necessarios)
2. [Carregamento dos dados](#carregamento-dos-dados)
4. [Comparação entre os dois conjuntos](#comparacao-entre-os-dois-conjuntos)
5. [Exportação dos dados unificados](#exportacao-dos-dados-unificados)

----

## Importação dos módulos e pacotes necessários <a id="importacao-dos-modulos-e-pacotes-necessarios"></a>

In [ ]:
# Pandas (https://pandas.pydata.org/)
import pandas as pd

# Locale (https://docs.python.org/3/library/locale.html)
import locale # Função de localização para formatar datas, número etc.

# Regular expressions (https://docs.python.org/3/library/re.html)
import re # Expressões regulares

# Datetime (https://docs.python.org/3/library/datetime.html)
import datetime as dt # Manipulação de data

---

## Carregamento dos dados

### Pará

In [ ]:
df_pa = pd.read_csv('./dados-tratados/20240221151519-dados_VCM_PA-pandas.csv', sep='|')

In [ ]:
# Visão geral do dados do Pará
df_pa.info()

In [ ]:
df_pa.head()

In [ ]:
print(df_pa.columns)

In [ ]:
# Contagem do número de variáveis
v = len(df_pa.columns)
# Contagem do número de registros
n = len(df_pa)

print(f'O conjunto de dados do Pará possui {n} registros e {v} variáveis.')

----

### Paraná

In [ ]:
df_pr = pd.read_csv('./dados-tratados/20240221211501-dados_VCM_PR-pandas.csv', sep='|')

In [ ]:
# Visão geral do dados do Paraná
df_pr.info()

In [ ]:
print(df_pr.columns)

In [ ]:
# Contagem do número de variáveis
v = len(df_pr.columns)
# Contagem do número de registros
n = len(df_pr)

print(f'O conjunto de dados do Paraná possui {n} registros e {v} variáveis.')

---

## Comparação entre os dois conjuntos  <a id="comparacao-entre-os-dois-conjuntos"></a>

Antes da comparação, é necessário adicionar as três colunas presentes nos dados do Paraná que não estão nos dados do Pará.

In [ ]:
df_pa['id_boletim'] = None
df_pa['vitima'] = None
df_pa['autor'] = None

In [ ]:
# Verificando se são as mesmas colunas
print(df_pa.columns == df_pr.columns)

Verificando o total da base, com a fusão dos dois conjuntos.

In [ ]:
len(df_pa) + len(df_pr)

## Exportação dos dados unificados <a id="exportacao-dos-dados-unificados"></a>

Criação de um dataframe único, com dados do Pará e do Paraná.

In [ ]:
df_unificado = pd.concat([df_pa, df_pr])

In [ ]:
df_unificado.head()

In [ ]:
len(df_unificado)

### Tratamentos adicionais

Verificação de melhorias percebidas durante a análise exploratória de dados.

---

#### Pontos de melhoria para o tratamento e formatação de dados

- Padronização das variáveis
  - Vítima - Faixa etária
  - Vítima - Grau de instrução --> SUPERIOR x TERCEIRO GRAU
  - Vítima - Profissão
  - Autor - Faixa etária
  - Autor - Grau de instrução --> SUPERIOR x TERCEIRO GRAU
  - Autor - Profissão
  - Grau de relacionamento

##### Faixa etária da Vítima

In [ ]:
# Definindo os intervalos e os rótulos das faixas etárias
intervals = [0, 11, 17, 24, 29, 34, 39, 44, 49, 64, 105]
labels = ['De 0 a 11 anos', 'De 12 a 17 anos', 'De 18 a 24 anos', 'De 25 a 29 anos', 'De 30 a 34 anos', 'De 35 a 39 anos', 'De 40 a 44 anos', 'De 45 a 49 anos', 'De 50 a 64 anos', 'Mais de 65 anos']
categories = labels + ['Desconhecida']

In [ ]:
# Vítima
df_unificado['vit_faixa_etaria'] = pd.cut(df_unificado['vit_idade'], bins=intervals, labels=labels, right=False)

# Adiciona a categoria 'Desconhecida' à lista de categorias da faixa etária
df_unificado['vit_faixa_etaria'] = pd.Categorical(df_unificado['vit_faixa_etaria'], categories=categories)

# Preenche os registros com valores nulos como 'Desconhecida'
df_unificado['vit_faixa_etaria'] = df_unificado['vit_faixa_etaria'].fillna('Desconhecida')

##### Faixa etária do Autor

In [ ]:
df_unificado['aut_faixa_etaria'] = pd.cut(df_unificado['aut_idade'], bins=intervals, labels=labels, right=False)

# Adiciona a categoria 'Desconhecida' à lista de categorias da faixa etária
df_unificado['aut_faixa_etaria'] = pd.Categorical(df_unificado['aut_faixa_etaria'], categories=categories)

# Preenche os registros com valores nulos como 'Desconhecida'
df_unificado['aut_faixa_etaria'] = df_unificado['aut_faixa_etaria'].fillna('Desconhecida')

##### Grau de instrução

In [ ]:
substituicoes = {
    'PRIMEIRO GRAU INCOMPLETO': 'ENSINO FUNDAMENTAL INCOMPLETO',
    'PRIMEIRO GRAU COMPLETO': 'ENSINO FUNDAMENTAL COMPLETO',
    'SEGUNDO GRAU INCOMPLETO': 'ENSINO MEDIO INCOMPLETO',
    'SEGUNDO GRAU COMPLETO': 'ENSINO MEDIO COMPLETO',
    'TERCEIRO GRAU INCOMPLETO': 'ENSINO SUPERIOR INCOMPLETO',
    'TERCEIRO GRAU COMPLETO': 'ENSINO SUPERIOR COMPLETO',
    'SUPERIOR INCOMPLETO': 'ENSINO SUPERIOR INCOMPLETO'
}

df_unificado['vit_grau_inst'] = df_unificado['vit_grau_inst'].replace(substituicoes)
df_unificado['aut_grau_inst'] = df_unificado['aut_grau_inst'].replace(substituicoes)

##### Profissões

In [ ]:
# Tornando como padrão os termos com mais registros
substituicoes = {
    'ADMINISTRADOR': 'ADMINISTRADOR DE EMPRESA',
    'ADVOGADO(A)': 'ADVOGADO',
    'AGRICULTOR(A)': 'AGRICULTOR',
    'APOSENTADO(A)': 'APOSENTADO',
    'CABELEIREIRO': 'CABELEIREIRA',
    'CAMAREIRO': 'CAMAREIRA',
    'COSTUREIRO(A),': 'COSTUREIRO',
    'COZINHEIRO(A),': 'COZINHEIRO',
    'ESTUDAMTE,': 'ESTUDANTE',
    'LAVRADOR(A)': 'LAVRADOR',
    'PEDAGOGO(A)': 'PEDAGOGA',
    'PEDAGOGO': 'PEDAGOGA',
    'PROMOTOR DE VENDAS': 'PROMOTORA DE VENDAS',
    'SECRETARIO(A)': 'SECRETARIA',
    'VENDEDOR(A)': 'VENDEDOR'
}

df_unificado['vit_profissao'] = df_unificado['vit_profissao'].replace(substituicoes)
df_unificado['aut_profissao'] = df_unificado['aut_profissao'].replace(substituicoes)

##### Grau de relacionamento

In [ ]:
# Tornando como padrão os termos com mais registros
substituicoes = {
    'nao informado': 'NAO INFORMADO',
    'Grau parentesco com o envolvido n.1-FILHO(A)': 'FILHO(A)',
    'Grau parentesco com o envolvido n.1-FILHA': 'FILHO(A)',
    'Grau parentesco com o envolvido n.1-EX_CONVIVENTE': 'EX-COMPANHEIRO(A)',
    'Grau parentesco com o envolvido n.1-EX_CONJUGE': 'EX-COMPANHEIRO(A)',
    'Grau parentesco com o envolvido n.1-IRMA': 'IRMAO(A)',
    'Grau parentesco com o envolvido n.1-NETO(A)': 'NETO(A)',
    'Grau parentesco com o envolvido n.1-EX CONVIVENTE': 'EX-COMPANHEIRO(A)',
    'Grau parentesco com o envolvido n.1-CASADO': 'COMPANHEIRO(A)',
	'Grau parentesco com o envolvido n.3-EX_CONVIVENTE': 'EX-COMPANHEIRO(A)',
	'Grau parentesco com o envolvido n.1-MAE': 'MAE',
	'Grau parentesco com o envolvido n.2-MAE': 'MAE',
	'Grau parentesco com o envolvido n.1-TIA': 'TIO(A)',
	'Grau parentesco com o envolvido n.1-CONCUNHADO(A)': 'CONCUNHADO(A)',
	'Grau parentesco com o envolvido n.1-NETA': 'NETO(A)',
	'Grau parentesco com o envolvido n.1-EX SOGRA': 'SOGRO(A)',
	'Grau parentesco com o envolvido n.1-EX ESPOSA': 'EX-COMPANHEIRO(A)',
}

df_unificado['grau_de_relacionamento'] = df_unificado['grau_de_relacionamento'].replace(substituicoes)

In [ ]:
# Verificação dos registros
registro = df_unificado.registros.value_counts().reset_index().rename(columns={'index': 'Registro', 'registros': 'Registro', 'count' : 'Total'}).reset_index(drop=True)

# Criando uma coluna com a porcentagem
registro['Porcentagem'] = registro.Total / registro.Total.sum()
registro

registro.style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

### Tratamento da coluna registros

In [ ]:
df_unificado['registros_bkp'] = df_unificado['registros']

In [ ]:
# Dicionário de categorias para 'registros'
categorias_registros = {
    'descumprir decisao judicial que defere medidas protetivas de urgencia':['descumprir decisao judicial que defere medidas protetivas de urgencia', 'descumprimento de medida protetiva'],
    'violencia domestica e familiar contra a mulher':['violencia domestica e familiar contra a mulher'],
    'lesao corporal':['lesao corporal'],
    'ameaca':['ameaca'],
    'estupro':['estupro'],
    'violencia sexual':['importunacao sexual', 'assedio sexual', 'intimidade sexual', 'assedio sexual', 'violacao sexual', 'abuso sexual', 'exploracao sexual', 'liberdade sexual', 'ato obsceno'],
    'tentativa de homicidio':['tentativa de homicidio'],
    'feminicidio':['feminicidio', 'femicidio', 'homicidio'],
}

# Função para substituir os valores na coluna 'registros'
def substituir_registro(valor):
    for key, values in categorias_registros.items():
        for value in values:
            if re.search(r'\b' + re.escape(value) + r'\b', valor.lower()):
                return key
    return 'outros'

In [ ]:
df_unificado['registros'] = df_unificado['registros'].apply(substituir_registro)

In [ ]:
df_unificado['registros'].value_counts()

In [ ]:
if 'registros_bkp' in df_unificado.columns:
    df_unificado.drop(columns=['registros_bkp'], inplace=True)

### Exportação

In [ ]:
df_unificado.to_csv('./dados-tratados/' + dt.datetime.now().strftime('%Y%m%d%H%M%S') + '-dados_unificados_VCM.csv', index=False, sep='|', encoding='utf-8')